# Hyperparameter Optimization

## Preliminaries
### Introduction
For the best performance for an ML-Model, we need to find the best hyperparameters. This file is for finding those. We want to evaluate
* Number of Neurons in each layer (for 20 Epochs, learning rate of 0.001 and lag of 60: best at 800)
* Number of epochs
* learning rate
* lag

For each of these parameters, we conduct an experiment of 50 values within chosen boundaries while all other parameters stay the same. For each of the parameters we safe the top 3 parameters and conduct a cross analysis out of those which combination works best.

### Load libraries

In [1]:
# homemade libraries
import Global_Functions as gf
import Neuronal_Networks as nn
import Data_Processing as dp

# Processing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# ML libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.callbacks import History

import time
timestr = time.strftime("%Y-%m-%d_%H-%M/")

### Load data

In [2]:
OPEN_FOLDER = '../Data/Preped_Data/'

In [3]:
ex_1 = gf.open_CSV_file('experiment_1_short.csv', OPEN_FOLDER)
ex_4 = gf.open_CSV_file('experiment_4_short.csv', OPEN_FOLDER)
ex_9 = gf.open_CSV_file('experiment_9_short.csv', OPEN_FOLDER)
ex_20 = gf.open_CSV_file('experiment_20_short.csv', OPEN_FOLDER)
ex_21 = gf.open_CSV_file('experiment_21_short.csv', OPEN_FOLDER)
ex_22 = gf.open_CSV_file('experiment_22_short.csv', OPEN_FOLDER)
ex_23 = gf.open_CSV_file('experiment_23_short.csv', OPEN_FOLDER)
ex_24 = gf.open_CSV_file('experiment_24_short.csv', OPEN_FOLDER)

In [4]:
experiments = [ex_1, ex_4, ex_9, ex_20, ex_21, ex_22, ex_23, ex_24]
names = ['1', '4', '9', '20', '21','22', '23', '24']

In [5]:
train = "20"
val = "21"
test = "22"

ex_train = ex_20
ex_val = ex_21
ex_test = ex_22

### Specify default parameters
Once again keep in mind, we change one of those parameters in every run through.

In [6]:
NEURONS = 64        #sensible boundaries are [8, 1024]
EPOCHS = 20          #sensible boundaries are [10,500]
LAG = 60            #sensible boundaries are [1,1000]
LEARNING_RATE = 0.1 #sensible boundaries are [0.001, 1]

In [7]:
image_path = "../Images/"
image_subfolder = image_path + "Hyperparameter_Optimization/"

gf.check_folder(image_path)
gf.check_folder(image_subfolder)

image_subfolder = image_subfolder + timestr
gf.check_folder(image_subfolder)

Folder already exists.
Folder already exists.
Creation of directory ../Images/Hyperparameter_Optimization/2021-05-13_11-37/ successful.


## Preprocessing of data

In [8]:
def prepare_data(data, lag = 1, all_inputs = True, include_power = False):
    length = len(data)
    input_scaled, scaler_input = dp.scale(data['input_voltage'])
    power_scaled, scaler_power = dp.scale(data['el_power'])
    
    scaler = [scaler_input, scaler_power]
    
    df = pd.DataFrame()
    
    for i in range(lag):
        df['input_voltage_delay_' + str(i)] = np.roll(input_scaled, i)[:length]
        df['el_power_delay_' + str(i)] = np.roll(power_scaled, i)[:length]
        
    # use either all lags or only the immediate input and the voltages just before.
    if all_inputs:
        filter_cols = [col for col in df if col.startswith('input_voltage_delay')]
        X = df[filter_cols]
    else:
        X = df[['input_voltage_delay_0', 'input_voltage_delay_' + str(lag-1)]]
    
    if include_power:
        X['el_power_delay_' + str(lag -1)] = df['el_power_delay_' + str(lag-1)]
        
    df['differences'] = df['el_power_delay_' + str(lag -1)] - df['el_power_delay_0']
    differences_scaled, scaler_differences = dp.scale(df['differences'])
    df['differences'] = differences_scaled
    scaler.append(scaler_differences)
    
    y = df[['el_power_delay_0', 'differences']]
    
    X = X.values
    X = X.reshape(X.shape[0],1 , X.shape[1])
    
    y = y.values
    y = y.reshape(y.shape[0], 1, y.shape[1])
    
    return scaler, X, y

In [9]:
def fit_lstm(X_train, y_train, X_val, y_val, batch_size, nb_epochs, neurons):
    history = History()

    model = Sequential()
    model.add(layers.LSTM(neurons, batch_input_shape = (batch_size, X_train.shape[1], X_train.shape[2]),
                          stateful = True, return_sequences = True))
    model.add(layers.LSTM(int(neurons/2), stateful = True))
    model.add(layers.Dense(2))
    model.compile(loss = 'MeanSquaredError', optimizer = 'adam', metrics = ['accuracy'],)
     
    hist = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = nb_epochs,
                    batch_size = batch_size, verbose = 1, shuffle = False, callbacks = [history])
    print('Training of model complete.')
    print("===========================================================")
    return model, hist

In [10]:
def plot_performance(history):
    # Plot the loss function
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    ax.plot(np.sqrt(history.history['loss']), 'r', label='train')
    ax.plot(np.sqrt(history.history['val_loss']), 'b' ,label='val')
    ax.set_xlabel(r'Epoch', fontsize=20)
    ax.set_ylabel(r'Loss', fontsize=20)
    ax.legend()
    ax.tick_params(labelsize=20)

    # Plot the accuracy
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    ax.plot(np.sqrt(history.history['accuracy']), 'r', label='train')
    ax.plot(np.sqrt(history.history['val_accuracy']), 'b' ,label='val')
    ax.set_xlabel(r'Epoch', fontsize=20)
    ax.set_ylabel(r'Accuracy', fontsize=20)
    ax.legend()
    ax.tick_params(labelsize=20)

In [11]:
def predictions(experiment, model, image_fol, batch_size = 1, specs = ""):
    scaler, X, y = prepare_data(experiment, lag = LAG)
    
    preds_scaled = model.predict(X, batch_size = batch_size)
    preds = scaler[1].inverse_transform(preds_scaled)[:,0]
    preds_diff = scaler[1].inverse_transform(scaler[2].inverse_transform(preds_scaled)[:,1])
    true = scaler[1].inverse_transform(y[:,0])
    
    fig = plt.figure(figsize = (15,10))
    plt.plot(true[:,0], color = gf.get_color("grey"), label = "True")
    plt.plot(preds, color = gf.get_color("green"), label = "Predictions")
    plt.ylabel('Electric power [W]', fontsize = 18)
    plt.xlabel('Time [sec]', fontsize = 18)
    plt.legend()
    plt.title('Predictions using loss function ' + specs, fontsize = 25)
    fig.tight_layout()
    plt.show()
    fig.savefig(image_fol + specs + "predictions.png")
    fig.savefig(image_fol + specs + "predictions.svg")
    
    fig.clear()
    
    fig = plt.figure(figsize = (15,10))
    plt.plot(true[:,1], color = gf.get_color("grey"), label = "True")
    plt.plot(preds_diff, color = gf.get_color("green"), label = "Predictions")
    plt.ylabel('Difference in Electric power [W]', fontsize = 18)
    plt.xlabel('Time [sec]', fontsize = 18)
    plt.legend()
    plt.title('Predictions of Differences using loss function ' + specs, fontsize = 25)
    fig.tight_layout()
    plt.show()
    fig.savefig(image_fol + specs + "preds_differences.png")
    fig.savefig(image_fol + specs + "preds_differences.svg")
    
    return scaler, X, y, preds_scaled, preds, preds_diff

In [12]:
def make_predictions(model, image_fol):
    investigate_experiments = [ex_train, ex_val, ex_test]
    investigate_names = ['train', 'val', 'test']
    for i in range(len(investigate_experiments)):
        predictions(investigate_experiments[i], model, image_fol, specs = "on {0} data".format(investigate_names[i]))

## Find optimal hyperparameters

### Neurons
We start with the number of neurons in each layer. Note, that this number only holds for the first value since the second layer is only half of the neurons. The output layer only contains two nodes.

In [13]:
random.seed(123)

In [14]:
image_folder = image_subfolder + "Epochs/"
gf.check_folder(image_folder)

Creation of directory ../Images/Hyperparameter_Optimization/2021-05-13_11-37/Epochs/ successful.


In [15]:
possible_epochs = []
for i in range(40):
    possible_epochs.append(random.randint(10, 300))
for i in range(10):
    possible_epochs.append(int(10*(i+1)))


In [16]:
def train_all_models(top = 3):
    all_models = list()
    all_histories = list()
    all_accs = list()
    all_accs_val = list()
    all_losses = list()
    all_losses_val = list()
    
    scaler_train, X_train, y_train = prepare_data(ex_train, lag = LAG)
    scaler_val, X_val, y_val = prepare_data(ex_val, lag = LAG)

    for e in possible_epochs:
        model, history = fit_lstm(X_train, y_train, X_val, y_val, 1, nb_epochs = e, neurons = NEURONS)
        all_models.append(model)
        all_histories.append(history)
        all_losses.append(history.history['loss'])
        all_losses_val.append(history.history['val_loss'])
        
    all_losses_val = [config_loss[-1] for config_loss in all_losses_val]
    models_losses_val_sorted = all_losses_val.copy()
    models_losses_val_sorted.sort(reverse=True)
    top_losses = models_losses_val_sorted[:top]
    top_models = list()
    top_histories = list()
    indices = list()
    for rank in top_losses:
        index = all_losses_val.index(rank)
        top_models.append(all_models[index])
        top_histories.append(all_histories[index])
        indices.append(index)
    return top_models, top_histories, indices

In [17]:
def analysze_top3(history):
    for h in history:
        plot_performance(h)

In [ ]:
top_3_models, top_3_histories, indices = train_all_models(top = 3)

Epoch 1/36
6495/6495 [==============================] - 28s 3ms/step - loss: 0.0206 - accuracy: 0.9892 - val_loss: 0.1484 - val_accuracy: 0.9641
Epoch 2/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0177 - accuracy: 0.9913 - val_loss: 0.1214 - val_accuracy: 0.9615
Epoch 3/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0176 - accuracy: 0.9910 - val_loss: 0.1780 - val_accuracy: 0.9677
Epoch 4/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0176 - accuracy: 0.9911 - val_loss: 0.1253 - val_accuracy: 0.9580
Epoch 5/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0177 - accuracy: 0.9907 - val_loss: 0.1447 - val_accuracy: 0.9657
Epoch 6/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0173 - accuracy: 0.9913 - val_loss: 0.0959 - val_accuracy: 0.9592
Epoch 7/36
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0175 - accuracy: 0.9897 - val_loss: 0.1696 - val_ac

6495/6495 [==============================] - 17s 3ms/step - loss: 0.0185 - accuracy: 0.9911 - val_loss: 0.0621 - val_accuracy: 0.9584
Epoch 21/147
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0175 - accuracy: 0.9907 - val_loss: 0.1301 - val_accuracy: 0.9689
Epoch 22/147
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0177 - accuracy: 0.9908 - val_loss: 0.2041 - val_accuracy: 0.9604
Epoch 23/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0173 - accuracy: 0.9913 - val_loss: 0.1886 - val_accuracy: 0.9550
Epoch 24/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0174 - accuracy: 0.9913 - val_loss: 0.2084 - val_accuracy: 0.9324
Epoch 25/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0179 - accuracy: 0.9911 - val_loss: 0.1941 - val_accuracy: 0.9544
Epoch 26/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0193 - accuracy: 0.9887 - val_loss: 0.3199 - val_a

Epoch 76/147
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0177 - accuracy: 0.9906 - val_loss: 0.2564 - val_accuracy: 0.9540
Epoch 77/147
6495/6495 [==============================] - 17s 3ms/step - loss: 0.0210 - accuracy: 0.9912 - val_loss: 0.2386 - val_accuracy: 0.9634
Epoch 78/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0207 - accuracy: 0.9913 - val_loss: 0.2311 - val_accuracy: 0.9646
Epoch 79/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0296 - accuracy: 0.9910 - val_loss: 0.1260 - val_accuracy: 0.9601
Epoch 80/147
6495/6495 [==============================] - 21s 3ms/step - loss: 0.0173 - accuracy: 0.9910 - val_loss: 0.1544 - val_accuracy: 0.9289
Epoch 81/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0252 - accuracy: 0.9901 - val_loss: 0.1031 - val_accuracy: 0.9621
Epoch 82/147
6495/6495 [==============================] - 20s 3ms/step - loss: 0.0175 - accuracy: 0.9903 - val_loss: 0

6495/6495 [==============================] - 16s 2ms/step - loss: 0.0190 - accuracy: 0.9909 - val_loss: 0.5298 - val_accuracy: 0.7749
Epoch 132/147
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0210 - accuracy: 0.9911 - val_loss: 0.2933 - val_accuracy: 0.8604
Epoch 133/147
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0243 - accuracy: 0.9909 - val_loss: 0.1792 - val_accuracy: 0.8619
Epoch 134/147
6495/6495 [==============================] - 17s 3ms/step - loss: 0.0195 - accuracy: 0.9903 - val_loss: 0.2611 - val_accuracy: 0.9677
Epoch 135/147
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0200 - accuracy: 0.9904 - val_loss: 0.2620 - val_accuracy: 0.8935
Epoch 136/147
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0197 - accuracy: 0.9899 - val_loss: 0.3256 - val_accuracy: 0.9358
Epoch 137/147
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0190 - accuracy: 0.9908 - val_loss: 0.4339 -

6495/6495 [==============================] - 18s 3ms/step - loss: 0.0177 - accuracy: 0.9907 - val_loss: 0.1516 - val_accuracy: 0.9652
Epoch 40/54
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0175 - accuracy: 0.9911 - val_loss: 0.2875 - val_accuracy: 0.9555
Epoch 41/54
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0185 - accuracy: 0.9907 - val_loss: 0.4538 - val_accuracy: 0.9597
Epoch 42/54
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0185 - accuracy: 0.9902 - val_loss: 0.1723 - val_accuracy: 0.9603
Epoch 43/54
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0176 - accuracy: 0.9908 - val_loss: 0.2895 - val_accuracy: 0.9635
Epoch 44/54
6495/6495 [==============================] - 16s 2ms/step - loss: 0.0180 - accuracy: 0.9914 - val_loss: 0.1077 - val_accuracy: 0.9623
Epoch 45/54
6495/6495 [==============================] - 15s 2ms/step - loss: 0.0176 - accuracy: 0.9913 - val_loss: 0.1124 - val_accurac

...

6495/6495 [==============================] - 19s 3ms/step - loss: 0.0195 - accuracy: 0.9906 - val_loss: 0.2639 - val_accuracy: 0.9627
Epoch 55/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0189 - accuracy: 0.9900 - val_loss: 0.3732 - val_accuracy: 0.9627
Epoch 56/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0178 - accuracy: 0.9910 - val_loss: 0.3164 - val_accuracy: 0.9651
Epoch 57/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0183 - accuracy: 0.9907 - val_loss: 0.1022 - val_accuracy: 0.9600
Epoch 58/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0189 - accuracy: 0.9907 - val_loss: 0.1921 - val_accuracy: 0.9369
Epoch 59/184
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0259 - accuracy: 0.9893 - val_loss: 0.1275 - val_accuracy: 0.9691
Epoch 60/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0189 - accuracy: 0.9904 - val_loss: 0.1736 - val_a

Epoch 110/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0183 - accuracy: 0.9909 - val_loss: 0.5409 - val_accuracy: 0.8727
Epoch 111/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0216 - accuracy: 0.9897 - val_loss: 0.3913 - val_accuracy: 0.8251
Epoch 112/184
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0198 - accuracy: 0.9914 - val_loss: 0.2769 - val_accuracy: 0.9692
Epoch 113/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0346 - accuracy: 0.9901 - val_loss: 0.3487 - val_accuracy: 0.9589
Epoch 114/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0195 - accuracy: 0.9900 - val_loss: 0.1607 - val_accuracy: 0.9283
Epoch 115/184
6495/6495 [==============================] - 18s 3ms/step - loss: 0.0183 - accuracy: 0.9911 - val_loss: 0.2188 - val_accuracy: 0.9590
Epoch 116/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0181 - accuracy: 0.9910 - val_

6495/6495 [==============================] - 19s 3ms/step - loss: 0.0188 - accuracy: 0.9914 - val_loss: 0.1091 - val_accuracy: 0.9621
Epoch 166/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0193 - accuracy: 0.9909 - val_loss: 0.1227 - val_accuracy: 0.9615
Epoch 167/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0201 - accuracy: 0.9897 - val_loss: 0.1552 - val_accuracy: 0.8851
Epoch 168/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0239 - accuracy: 0.9887 - val_loss: 0.1452 - val_accuracy: 0.9635
Epoch 169/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0204 - accuracy: 0.9916 - val_loss: 0.1580 - val_accuracy: 0.9646
Epoch 170/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0190 - accuracy: 0.9903 - val_loss: 0.2118 - val_accuracy: 0.8785
Epoch 171/184
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0186 - accuracy: 0.9907 - val_loss: 0.3713 -

6495/6495 [==============================] - 19s 3ms/step - loss: 0.0174 - accuracy: 0.9907 - val_loss: 0.1603 - val_accuracy: 0.9592
Training of model complete.
Epoch 1/91
6495/6495 [==============================] - 23s 3ms/step - loss: 0.0231 - accuracy: 0.9857 - val_loss: 0.2013 - val_accuracy: 0.8470
Epoch 2/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0173 - accuracy: 0.9911 - val_loss: 0.1989 - val_accuracy: 0.9595
Epoch 3/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0176 - accuracy: 0.9910 - val_loss: 0.2724 - val_accuracy: 0.8500
Epoch 4/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0173 - accuracy: 0.9911 - val_loss: 0.1847 - val_accuracy: 0.9547
Epoch 5/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0171 - accuracy: 0.9914 - val_loss: 0.1959 - val_accuracy: 0.8670
Epoch 6/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0172 - accuracy: 0.9913 - val_loss

6495/6495 [==============================] - 19s 3ms/step - loss: 0.0182 - accuracy: 0.9910 - val_loss: 0.2225 - val_accuracy: 0.9581
Epoch 56/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0184 - accuracy: 0.9905 - val_loss: 0.1202 - val_accuracy: 0.9610
Epoch 57/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0179 - accuracy: 0.9906 - val_loss: 0.2161 - val_accuracy: 0.8947
Epoch 58/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0188 - accuracy: 0.9912 - val_loss: 0.0968 - val_accuracy: 0.9440
Epoch 59/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0179 - accuracy: 0.9912 - val_loss: 0.2899 - val_accuracy: 0.9647
Epoch 60/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0177 - accuracy: 0.9909 - val_loss: 0.2096 - val_accuracy: 0.9604
Epoch 61/91
6495/6495 [==============================] - 19s 3ms/step - loss: 0.0229 - accuracy: 0.9912 - val_loss: 0.0653 - val_accurac

In [ ]:
analysze_top3(top_3_histories)

In [ ]:
indices

In [ ]:
for i in indices:
    print(possible_epochs[i])

We found in our case the top 3 performing models had a 16, 795 or 800 neurons. Since 795 and 800 are quite similar, we will only compare 16 and 800 in the cross-optimization.